# Modelado Probabilístico e Inferencia

![BN](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/SimpleBayesNet.svg/1920px-SimpleBayesNet.svg.png)

> Ya que vimos las ventajas del enfoque Bayesiano, hablaremos un poco acerca de cómo definir modelos probabilísticos a través de Redes Bayesianas (BN - Bayesian Networks).

> **Objetivos:**
> - Comprender qué es una red Bayesiana, como definirla, y qué tipo de distribuciones codifica.
> - Establecer inferencias probabilísticas sobre una red Bayesiana.

> **Referencias:**
> 
> - Probabilistic Graphical Models: Principles and Techniques, By Daphne Koller and Nir Friedman. Cap. 2.
> - Bayesian Methods for Machine Learning course, HSE University, Coursera.

## 1. Redes Bayesianas

Una forma muy conveniente de definir modelos probabilísticos es a través de las llamadas **Redes Bayesianas**.

> *Definición.* Una **Red Bayesiana** es un grafo (acíclico) dirigido donde los nodos representan variables aleatorias y los arcos representan impacto directo de una variable sobre otra.

Por ejemplo, podemos considerar la siguiente Red Bayesiana sobre las variables $L$ (lluvia), $R$ (riego), $M$ (pasto mojado):

![bn1](figures/BN1.png)

Definimos los nodos padres de un nodo $X$ como todos los nodos de los que nazca un arco apuntando hacia $X$. En el ejemplo:

- $Pa(M)=\{R, L\}$
- $Pa(R)=\{L\}$
- $Pa(L)=\{\}$

La probabilidad conjunta sobre todas las variables en una red Bayesiana se factoriza como:

$$
P(X_1, X_2, \dots, X_n) = \prod_{k=1}^{n}P(X_k | Pa(X_k)).
$$

De modo que para nuestro ejemplo, el modelo sería:

$$
P(L, R, M) = P(L) P(R | L) P(M | L, R)
$$

## 2. Inferencia en Redes Bayesianas

Un caso que ya estamos listos para enfrentar es el ejemplo de una alarma antirobo.

En este caso, suponemos que instalamos una alarma antirrobo en nuestras casas la cual se activa mediante un sensor de movimiento. Si un ladrón entra a nuestra casa, el sensor detectará el movimiento y la alarma nos enviará una notificación.

Sin embargo, si hay un terremoto, la alarma puede mandarnos una notificación falsa. Adicionalmente, dependiendo de la magnitud del terremoto, habrán reportes de él (aunque la mayoría de terremotos quedan registrados en los institutos sísmicos de cada país, no todos merecen un reporte inmediato, especialmente si son de bajas magnitudes).

De modo que un modelo gráfico considerando las variables $L$ (ladrón), $A$ (alarma), $T$ (terremoto) y $R$ (reporte en la radio) es:

![bn2](figures/BN2.png)

¿Cómo sería la distribución conjunta sobre las variables $L, A, T$ y $R$?

$$
P(L,A,T,R) = ?
$$

Tenemos entonces la siguiente información:

- $P(L=1) = 0.001$ (una casa de cada mil es robada)
- $P(T=1) = 0.01$ (un terremoto cada 100 días)

Las probabilidades de que la alarma se active (dadas por el proveedor de alarmas) son:

| $L$ | $T$ | $P(A=1 | L, T)$ |
| --- | --- | --------------- |
| $0$ | $0$ | $0$             |
| $0$ | $1$ | $0.1$           |
| $1$ | $0$ | $1$             |
| $1$ | $1$ | $1$             |

Las probabilidades de que un terremoto se reporte en la radio es:

| $T$ | $P(R=1 | T)$    |
| --- | --------------- |
| $0$ | $0$             |
| $1$ | $0.5$           |

Supongamos que nos llega entonces una notificación de la alarma a nuestro celular. Queremos saber si en efecto la alarma se debe a un ladrón, o no. Esto es, queremos averiguar

$$
P(L=1|A=1)=?
$$

<font color=green> Resolver en el pizarrón </font>

De modo que te subes al coche y te diriges camino a tu casa, pero al encender la radio escuchas que hubo un terremoto. Ahora, lo que queremos averiguar es:

$$
P(L=1|A=1, R=1)=?
$$

<font color=green> Resolver en el pizarrón </font>

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>